In [1]:
from pandas import ExcelFile, read_excel, MultiIndex, DataFrame, read_csv,concat
from numpy import nonzero, isnan, nan, vectorize, average, int32, zeros, pad, array
from time import sleep
from datetime import datetime

from scipy.ndimage import maximum_filter


def read_DB(db_path):
    '''
    function for reading database and parse it to dictionary of dataframes
    nameOrigin is used for indexing and presenting the database entries in a understandable way for the user
    '''
    db_sh = ExcelFile(db_path)
    sheets = db_sh.sheet_names
    db = read_excel(db_path, sheet_name= sheets, index_col= 0)

    for col in sheets:
        
        if col == 'Name':
            db[col]['nameOrigin'] = db[col]['Name'].astype(str) + ', ' + db[col]['Origin'].astype(str)
        elif col == 'References': 
            db[col]['authorYear'] = db[col]['Author'].astype(str) + ', ' + db[col]['Year'].astype(str)
        elif col == 'Country':
            db[col]['nameOrigin'] = db[col]['Country'].astype(str) + ', ' + db[col]['City'].astype(str)  
        elif col == 'Region':
            pass
        elif col == 'Spartacus Material':
            db[col]['nameOrigin'] = db[col]['Name'].astype(str) + '; ' + db[col]['Color'].astype(str) + '; ' + db[col]['Origin'].astype(str)    
        # Calculate U-values for roof and wall new columns u_value_wall and u_value_roof
        
        elif col == 'Spartacus Surface':
            db[col]['nameOrigin'] = db[col]['Name'].astype(str) + ', ' + db[col]['Origin'].astype(str)
                # Filter rows where Surface is 'Buildings'
        
            buildings = db['Spartacus Surface'][db['Spartacus Surface']['Surface'] == 'Buildings']

            # Calculate resistances and U-values
            for prefix in ['w', 'r']:

                if prefix == 'w':
                    pr = 'wall'
                else:
                    pr = 'roof'
                materials = buildings[[f'{prefix}{i}Material' for i in range(1, 6)]].values
                thicknesses = buildings[[f'{prefix}{i}Thickness' for i in range(1, 6)]].values

                thicknesses[isnan(thicknesses)] = 0

                for i in range(0,5):
                    materials[isnan(materials)] = materials[nonzero(isnan(materials))[0], nonzero(isnan(materials))[1]-1]


                thermal_conductivities = vectorize(lambda x: db['Spartacus Material'].loc[x, 'Thermal Conductivity'])(materials)


                resistances = thicknesses / thermal_conductivities
                resistance_bulk = resistances.sum(axis=1)

                u_values = 1 / resistance_bulk

                db['Spartacus Surface'].loc[buildings.index, f'u_value_{pr}'] = u_values

            # Calculate albedo and emissivity
            for prop in ['Albedo', 'Emissivity']:
                for prefix, pr in zip(['w', 'r'], ['wall', 'roof']):

                    material_col = f'{prefix}1Material'
                    db['Spartacus Surface'].loc[buildings.index, f'{prop.lower()}_{pr}'] = db['Spartacus Material'].loc[buildings[material_col], prop].values
        
        elif col == 'Profiles':
            # Normalise traffic and energy use profiles to ensure that average of all columns = 1
            normalisation_rows = db[col][(db[col]['Profile Type'] == 'Traffic') | (db[col]['Profile Type'] == 'Energy use')]
            cols = list(range(24))
            normalisation_rows_index = list(normalisation_rows.index)

            # # # Calculate the sum of the values for each row
            sums = db[col].loc[normalisation_rows_index, cols].sum(axis=1)

            # Avoid division by zero by replacing zero sums with NaN
            sums.replace(0, nan, inplace=True)

            # # Calculate the scaling factor to make the sum equal to the number of columns (24)
            scaling_factors = 24 / sums

            # Scale the values
            db[col].loc[normalisation_rows_index, cols] = db[col].loc[normalisation_rows_index, cols].multiply(scaling_factors, axis=0)
            
            # Create unique name
            db[col]['nameOrigin'] = db[col]['Name'].astype(str)  +  ', ' + db[col]['Day'].astype(str) +  ', ' + db[col]['Country'].astype(str) + ', ' + db[col]['City'].astype(str) 

        else:
            # Standard
            db[col]['nameOrigin'] = db[col]['Name'].astype(str) + ', ' + db[col]['Origin'].astype(str)

    db_sh.close() # trying this to close excelfile

    return db


In [11]:
db_dict = read_DB('suews_prepare_database/Input/database.xlsx')

In [1]:
parameter_dict = {'TrafficRate_WD': 60240493, 'TrafficRate_WE': 60240492, 'SnowClearingProfWD': 60240006, 'SnowClearingProfWE': 60240007, 'AnthropogenicCode': 53240050, 'IrrigationCode': 61240005, 'WaterUseProfManuWD': 60240000, 'WaterUseProfManuWE': 60240001, 'WaterUseProfAutoWD': 60240000, 'WaterUseProfAutoWE': 60240001, 'EnergyUseProfWD': 60240492, 'EnergyUseProfWE': 60240492, 'ActivityProfWD': 60240032, 'ActivityProfWE': 60240033, 'PopProfWD': 60240032, 'PopProfWE': 60240033, 'Paved': 20240016, 'Buildings': 20240013, 'Bare Soil': 20240025, 'Grass': 24240028, 'Evergreen Tree': 24240026, 'Deciduous Tree': 24240027, 'SoilTypeCode': 22240040, 'Water': 25240003, 'SnowCode': 23240006, 'Conductance': 44240002, 'TCritic_Heating_WD': 7, 'TCritic_Heating_WE': 7, 'TCritic_Cooling_WD': 7, 'TCritic_Cooling_WE': 7, 'BaseT_HC': 18.2, 'Traffic_multiplier_OSM': '{"Tertiary":16.0,"Primary":110.8, "Motorway":159.2, "Motorway Road":36.0,"Secondary":52.1,"Residential, Living street":6.6}', 'Vegetation Growth': 35240002, 'nameOrigin': 'Anguilla, LUCY'}

In [5]:
settings_dict = {'poly_field': 'ID', 'Metfile_path': ('C:/Users/xbacos/OneDrive - University of Gothenburg/Artikel_4/MET/Kb_2017_data_60.txt', 'All Files (*)'), 'start_DLS': 85, 'end_DLS': 302, 'LCF_from_file': True, 'LCFfile_path': ['C:/Users/xbacos/OneDrive - University of Gothenburg/Artikel_4/GIS/morph/lc_LCFG_isotropic.txt'], 'IMP_from_file': True, 'IMPfile_path': ['C:/Users/xbacos/OneDrive - University of Gothenburg/Artikel_4/GIS/morph/imp_IMPGrid_isotropic.txt'], 'IMP_z0': None, 'IMP_zd': None, 'IMP_fai': None, 'IMPveg_from_file': True, 'IMPvegfile_path': ['C:/Users/xbacos/OneDrive - University of Gothenburg/Artikel_4/GIS/morph/veg_IMPGrid_isotropic.txt'], 'IMPveg_fai_eve': None, 'IMPveg_fai_dec': None, 'pop_density': '_popsum', 'plugin_dir': 'C:\\Users/xbacos/AppData/Roaming/QGIS/QGIS3\\profiles\\default/python/plugins\\suews_prepare_database', 'output_dir': ['C:/Users/xbacos/OneDrive - University of Gothenburg/Artikel_4/OUT/O_nov'], 'file_code': 'fa', 'utc': 0, 'checkBox_twovegfiles': None, 'IMPvegfile_path_dec': None, 'IMPvegfile_path_eve': None, 'pop_density_day': 'left', 'daypop': 0, 'region_str': 'Caribbean', 'country_str': 'Anguilla, LUCY', 'checkBoxTypologies': 1, 'heightMethod': 3, 'vertheights': '10, 20', 'nlayers': 3, 'skew': 2, 'ss_dir': 'C:/Users/xbacos/OneDrive - University of Gothenburg/Artikel_4/GIS/morph', 'spartacus': 1, 'Paved': 'City centre , Beijing', 'Buildings': 'City centre , Beijing', 'Bare Soil': 'Central London, Ward et al. (2016), London, UK', 'Evergreen Tree': 'Ward et al. (2016), London, UK', 'Decidous Tree': 'Ward et al. (2016), London, UK', 'Grass': 'Ward et al. (2016), London, UK', 'AnthropogenicCode': 'Central London, London, UK', 'TrafficRate_WD': 'LUCY, Weekday, Anguilla, nan', 'TrafficRate_WE': 'LUCY, Weekend, Anguilla, nan', 'SnowClearingProfWD': 'Snow removed daytime, Weekday, Finland, Helsinki', 'SnowClearingProfWE': 'Snow removed daytime, Weekend, Finland, Helsinki', 'ActivityProfWD': 'LUCY, Weekday, Anguilla, nan', 'ActivityProfWE': 'LUCY, Weekend, Anguilla, nan', 'PopProfWD': 'LUCY, Weekday, Anguilla, nan', 'PopProfWE': 'LUCY, Weekend, Anguilla, nan', 'EnergyUseProfWD': 'LUCY, Weekend, Anguilla, nan', 'EnergyUseProfWE': 'LUCY, Weekend, Anguilla, nan', 'SnowRemovalWD': '[old code: 11], Weekday, United States of America, Los Angeles', 'SnowRemovalWE': '[old code: 12], Weekend, United States of America, Los Angeles', 'WaterUseProfAutoWD': '[old code: 11], Weekday, United States of America, Los Angeles', 'WaterUseProfAutoWE': '[old code: 12], Weekend, United States of America, Los Angeles'}

In [19]:
# Save Profiles
profiles = ['TrafficRate_WD','TrafficRate_WE', 'EnergyUseProfWD','EnergyUseProfWE','ActivityProfWD','ActivityProfWE','PopProfWD','PopProfWE', 'SnowClearingProfWD', 'SnowClearingProfWE','WaterUseProfManuWD','WaterUseProfManuWE','WaterUseProfAutoWD','WaterUseProfAutoWE']        
profiles_dict = {}

for i in profiles:
    try:
        profiles_dict[i] = {
            'profileCode' : parameter_dict[i],
            
        }
    except:
        print(i)

parameter_dict[i]

60240001

In [20]:
db_dict['Profiles'].loc[parameter_dict[i], ['Profile Type', 'Day', 'nameOrigin']]

Profile Type                                Water use (automatic)
Day                                                       Weekend
nameOrigin      [old code: 12], Weekend, United States of Amer...
Name: 60240001, dtype: object

In [18]:
db_dict['Profiles'].loc[parameter_dict[i], ['Profile Type', 'Day', 'nameOrigin']].agg(' '.join)

C:\Users\xbacos\AppData\Local\Temp\ipykernel_22388\4217028195.py:1: FutureWarning: using <built-in method join of str object at 0x00007FF94D8E34D8> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  db_dict['Profiles'].loc[parameter_dict[i], ['Profile Type', 'Day', 'nameOrigin']].agg(' '.join)


Profile Type            W a t e r   u s e   ( a u t o m a t i c )
Day                                                 W e e k e n d
nameOrigin      [ o l d   c o d e :   1 2 ] ,   W e e k e n d ...
Name: 60240001, dtype: object

In [ ]:
heightIntervals = [0.0, 4.0, 8.0, 12.0, 16.0, 20.0, 24.0, 28.0, 32.0, 36.0, 40.0, 44.0, 48.0, 52.0, 56.0, 57.0]
interval = 4

def check_intervals(lst, interval):
    for i in range(1, len(lst)):
        if lst[i] - lst[i-1] != interval:
            print('fales')
    

is_correct_interval = check_intervals(heightIntervals, interval)

fales


In [ ]:
heightIntervals = [0.0, 4.0, 8.0, 12.0, 16.0, 20.0, 24.0, 28.0, 32.0, 36.0, 40.0, 44.0, 48.0, 52.0, 56.0, 57.0]
interval = 4

def check_intervals(heightIntervals, interval):
    for i in range(1, len(heightIntervals)):
        if heightIntervals[i] - heightIntervals[i-1] != interval:
            print('fales')


In [49]:
spartacus_error_dict = {}

fa = {1 :'12312'}
spartacus_error_dict = spartacus_error_dict | fa
ba = {2 :'12sdaf312'}
spartacus_error_dict = spartacus_error_dict | ba

if spartacus_error_dict:
    print('tes')

tes


In [ ]:
spartacus_error_dict

error_string = ''
for keys in list(spartacus_error_dict.keys()):
    error_string = error_string  + '\n Error in Grid: '#{str(keys)} : {spartacus_error_dict[keys]}'

In [48]:
error_string  + f'\nError in Grid: {str(keys)} : {spartacus_error_dict[keys]}'

'\\n Error in Grid: \\n Error in Grid: \nError in Grid: 2 : 12sdaf312'

In [33]:
spartacus_error_dict = {}


fa = {1 :'12312'}
spartacus_error_dict = spartacus_error_dict | fa

if not spartacus_error_dict:
    print('empty')

In [2]:
import pandas as pd

In [ ]:
'C:/Users/xbacos/OneDrive - University of Gothenburg/Artikel_4/MET/Kb_2017_data_60.txt'